In [ ]:
import torch
from torch import nn
from model import VAE
from torch.distributions import Normal
import torch.nn.functional as F


In [1]:
from utils import save_kmeans_file

In [2]:
save_kmeans_file(2)

Fitting model on  (600000, 64, 64, 1) <class 'numpy.ndarray'> 0 255


(array([[0.00380249],
        [0.88080809]]),
 [0.05190242513020833, 0.05229036458333333, 0.05212353515625],
 [0.22183003267324153, 0.22261195465445155, 0.22227611711352413])

In [ ]:
in_channels = 1
out_channels = 1
intermediate_channels = 32
z_dimensions = 16
sigma_decoder = 0.1
q_z = Normal(torch.tensor(0.), torch.tensor(1.))

vae_with_no_kl = VAE(in_channels, intermediate_channels, 1, z_dimensions, pixelcnn=False, kl=0, mmd=0)
vae_with_kl = VAE(in_channels, intermediate_channels, 1, z_dimensions, pixelcnn=False)
vae_with_mmd = VAE(in_channels, intermediate_channels, 1, z_dimensions, pixelcnn=False, kl=0, mmd=1, require_rsample=False)

pixelvae_with_no_kl = VAE(in_channels, intermediate_channels, 256, z_dimensions, pixelcnn=True,kl=0, sigma_decoder=0, require_rsample=False)
pixelvae_with_kl = VAE(in_channels, intermediate_channels, 256, z_dimensions, pixelcnn=True, kl=1, mmd=0, sigma_decoder=0)
pixelvae_with_mmd = VAE(in_channels, intermediate_channels, 256, z_dimensions, pixelcnn=True, kl=0, mmd=1, require_rsample=False, sigma_decoder=0)


In [ ]:
input_1 = torch.randn(10,1,64,64)
target_1 = torch.randn(10,1,64,64)
target_255 = torch.randint(0, 256, (10,64,64)).long()

In [ ]:
mu, logvar, encoding, reconstruction = vae_with_no_kl(input_1)
vae_with_no_kl.loss(target_1, mu, logvar, encoding, reconstruction)
print(-Normal(reconstruction, sigma_decoder).log_prob(target_1).sum() / target_1.shape[0])

In [ ]:
mu, logvar, encoding, reconstruction = vae_with_kl(input_1)
q_z_x = Normal(mu, (0.5 * logvar).exp())
print(vae_with_kl.loss(target_1, mu, logvar, encoding, reconstruction))
print((-Normal(reconstruction, sigma_decoder).log_prob(target_1).sum())/target_1.shape[0] +
     (torch.distributions.kl.kl_divergence(q_z_x, q_z).sum())/target_1.shape[0])

In [ ]:
mu, logvar, encoding, reconstruction = vae_with_mmd(input_1)
mmd = vae_with_mmd.compute_mmd(torch.randn(input_1.shape[0], mu.shape[1]), encoding.view(-1, mu.shape[1]))
print(vae_with_mmd.loss(target_1, mu, logvar, encoding, reconstruction))
print((-Normal(reconstruction, sigma_decoder).log_prob(target_1).sum() + mmd)/input_1.shape[0])

In [ ]:
input_1 = torch.randn(10,1,64,64)
target_1 = torch.randn(10,1,64,64)
target_255 = torch.randint(0, 256, (10,64,64)).long()

mu, logvar, encoding, reconstruction = pixelvae_with_no_kl(input_1)
print(pixelvae_with_no_kl.loss(target_255, mu, logvar, encoding, reconstruction) -
F.cross_entropy(reconstruction, target_255, reduction='none').sum() / target_255.shape[0])

In [ ]:
mu, logvar, encoding, reconstruction = pixelvae_with_kl(input_1)
q_z_x = Normal(mu, (0.5 * logvar).exp())

print(pixelvae_with_kl.loss(target_255, mu, logvar, encoding, reconstruction))
print((F.cross_entropy(reconstruction, target_255, reduction='sum'))/target_1.shape[0] +
     (torch.distributions.kl.kl_divergence(q_z_x, q_z).sum())/target_1.shape[0])

In [ ]:
mu, logvar, encoding, reconstruction = pixelvae_with_mmd(input_1)
mmd = pixelvae_with_mmd.compute_mmd(torch.randn(input_1.shape[0], mu.shape[1]), encoding.view(-1, mu.shape[1]))

print(pixelvae_with_mmd.loss(target_255, mu, logvar, encoding, reconstruction))
print(F.cross_entropy(reconstruction, target_255, reduction='sum') / target_255.shape[0]) 